In [1]:
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
data_path = "/Users/trinakarmakar/anaconda2/data/fastai/redux/"
model_path = data_path+'models/'
print(model_path)
if not os.path.exists(model_path):
    os.mkdir(model_path)

/Users/trinakarmakar/anaconda2/data/fastai/redux/models/


In [3]:
batch_size = 64

Vgg model uses dropout and what we are seeing is validation accuracy greater than training accuracy, that is not desirable.
This could mean that our model is underfitting which we need to avoid. We suspect that it could be because of 
Regularizing model little too much and we need to fix that by working on the regularization from scratch.

Approach:We will take vgg model, split it into cnv layers and dense layers. We will pre train conv layers. 
Then we will remove dropout from dense layer by setting p = 0.0 and then build the new model and train them.

In [4]:
model = vgg_ft(2)

In [5]:
#and load our tuned weights

model.load_weights(model_path+'finetune3.h5')

In [6]:
layers = model.layers

In [7]:
last_conv_idx = [index for index, layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [8]:
last_conv_idx

30

In [9]:
layers[last_conv_idx]

In [18]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
fc_layers = layers[last_conv_idx+1:]

In [11]:
#keras returns classes as single columns needs to convert it to onehot
def one_hot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1, 1)).todense())

In [12]:
batches = get_batches(data_path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(data_path+'valid', shuffle=False, batch_size=batch_size)
train_classes = batches.classes
val_classes = val_batches.classes

train_labels = one_hot(train_classes)
val_labels = one_hot(val_classes)

Found 22750 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [13]:
batches.class_indices

{'cats': 0, 'dogs': 1}

In [27]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

In [ ]:
train_features = conv_model.predict_generator(batches, batches.nb_sample)

In [ ]:
save_array(model_path + 'train_conv_layer_features.bc', train_features)
saver_array(model_path + 'val_conv_layer_features.bc', val_features)

In [ ]:
train_features.shape

In [14]:
#Since we are removing dropout we need to half the weights
def proc_weights(layer):
    return [o/2 for o in layer.get_weights()]

In [15]:
optimizer = RMSprop(lr = 0.01, rho=0.7) #fine tuned model needs to be updated very slowly

In [24]:
#Build new fc model
def get_fc_model():
    
    model = Sequential([MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
                       Flatten(),
                       Dense(4096, activation='relu'),
                       Dropout(0.),
                       Dense(4096, activation='relu'),
                       Dropout(0.),
                       Dense(2, activation='softmax')
                       ])
    for l1, l2 in zip(model.layers, fc_layers):
        l1.set_weights(proc_weights(l2))
    
    model.compile(optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
fc_model = get_fc_model()

In [26]:
fc_model.fit(train_features, train_labels, nb_epoch=2, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

NameError: name 'train_features' is not defined

In [ ]:
fc_model.save_weights(model_path+'no_dropout.h5')

In [ ]:
fc_model.load_weights(model_path+'no_dropout.h5')

Now we created a model that overfits we can take number of steps to reduce overfitting:
    1. Add more data
    2. Use data augmentation
    3. Get an architecture that generarilzies well
    4. Add regularization 
    4. Recuce architecture complexity

In [ ]:
#getting more data often difficult and sometime impossible, then we move to option 2 where we can generate data using
#data augmentation

gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                              height_shift_range=0.1, shear_range=0.15, zoom_range=0.1,
                              channel_shift_range=10, horizantal_flip=True, dim_ordering='tf')

In [ ]:
#lets look at one image and how it changes
img = np.expand.dims(ndimage.imread(data_path+'test_image.jpg'),0)
aug_iter = gen.flow(img) #create batches from this image

In [ ]:
aug_imgs = [next(aug_iter)[0].astype(np.unit8) for i in range(8)]

In [ ]:
plt.imshow(img[0])

In [ ]:
plots(aug_imgs, (20,7), 2)

### Adding augmented data with small amout to see if this removes overfitting problem, we will use same method as lesson 2 to fine tune but with fit generator with augmented data

In [ ]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizantal_flip=True)

In [ ]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
#We don't augment or shuffle the validation set
val_batches = get_batches(path+'valid', batch_size=batch_size)

In [ ]:
fc_model = get_fc_model()

In [ ]:
for layer in conv_model.layers:
    layer.trainable = False
conv_model.add(fc_model)

In [ ]:
#compile, train and save model as usual
conv_model.compile(optimizer=opt, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_model.fit_generhttp://localhost:8888/notebooks/fastai_lesson3.ipynb#ator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8, validation_data = val_batches,
                        nb_val_smaples = val_batches.nb_sample)

In [ ]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                        nb_val_smaples = val_batches.nb_sample)

In [ ]:
conv_model.save_weights(model_path+'aug1.h5')
conv_model.load_weights(model_path+'aug1.h5')

### Batchnormalization - Normalizing activation in neurons so that activation remains in the same scale is called batchnormalization. Batchnormalization improves speed of training process by avoiding gradient to become very large or very small on both case training fails or become very slow. Because of this normalization problem in choosing not so great initial parameters are overcome to some extent and also it helps to reduce some outlying inputs to over influence the training, so tends to reduce overfitting

In [ ]:
#We will now add batchnorm and dropout layers
conv_layers[-1].output_shape[1:]

In [ ]:
def load_fc_weights_from_vgg16bn(model):
    #load weights from the dense layers of the vgg16bn model
    from vgg16bn import Vgg16BN
    vgg16bn = Vgg16BN()
    _, fc_layers = split_at(vgg16bn.model, Convlution2D)
    copy_weights(fc_layers, model.layers)

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='')
        BatchNormalization(),
        Dropout(p),
        Dense(1000, activation='softmax')
    ]

In [ ]:
p = 0.5

In [ ]:
bn_model=Sequential(get_bn_layers(p))

In [ ]:
load_fc_weights_from_vgg16bn(bn_model)

In [ ]:
def proc_weights(layers, prev_p, new_p):
    scal = (1 - prev_p) / (1 - new_p)
    return (o*scal for o in layer.get_weights)

In [ ]:
for l in bn_model.layers:
    if type(l) == Dense:
        l.set_weights(proc_weights(1, 0.5, 0.6))

In [ ]:
bn_model.pop()
for layer in bn_model.layers:
    layer.trainable = False

In [ ]:
bn_model.add(Dense(2, activation=False))

In [ ]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
bn_model.fit(train_features, train_labels, nb_epoch=8, validation_data = (val_features, val_labels))

In [ ]:
bn_model.save_weights(model_path+'bn.h5')
bn_model.load_weights(model_path+'bn.h5')

In [ ]:
bn_layers = get_bn_layers(0.5)
bn_layers.pop() #remove previous softmax layer and add a new one
bn_layers.append(Dense(2, (activation='softmax'))

We add all the convolutional layers in the final model for which it is already trained and we saved the weights
Then we add rest of the layers starting with a maxpool layer and sequence of flatten, dense, dropout and batchnorm
layers

In [ ]:
final_model = Sequential(conv_layers)
for layer in final_model.layers:
    layer.trainable = False
for layer in bn_layers:
    final_model.add(layer)

In [ ]:
#take the weights from bn_model which we trained here and set it to the bn layers
for l1, l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights)

In [ ]:
final_model.compile(optimizer=Adam(), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1,
                         validation_data = val_batches, nb_val_smaples = val_batches.nb_smaple)

In [ ]:
final_model.save_weights(model_path+'final1.h5')

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4,
                         validation_data=val_batches. nb_val_samples = val_batches.nb_sample)

In [ ]:
final_model.save_weights(model_path+'final2.h5')

In [ ]:
final_model.optimizer.lr = 0.001

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4,
                         validation_data=val_batches, nb_val_samples = val_batches.nb_sample)

In [ ]:
bn_model.save_weights(model_path+'final3.h5')